In [337]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [338]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [339]:
import torch

In [340]:
from torch import nn

In [341]:
df = pd.read_csv('Lab2_Dataset.csv')
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x68,x69,x70,x71,x72,x73,x74,x75,x76,y
0,1.704359,0.518496,0.167880,1.413905,2.383612,2.240500,2.282446,-0.083431,0.034095,-2.511093,...,-0.607536,-0.477308,-0.297182,2.195598,1.235382,0.854542,0.228877,-0.245788,0.815149,1
1,0.011969,2.267140,0.588663,-0.407317,-0.109885,-0.145653,-0.139028,-0.083431,-0.087286,1.013473,...,0.505555,-0.477308,-0.297182,-0.885894,-1.172618,-1.370928,-0.880668,-0.245788,-0.919861,0
2,-0.302977,-0.621716,-0.674528,0.958600,-0.125325,-0.066033,-0.058229,-0.083431,-0.087286,-0.205345,...,-0.607536,2.095085,-0.297182,2.195598,0.633382,0.536618,1.338422,-0.245788,2.405900,0
3,-0.642308,-0.954021,-0.253745,-0.407317,-0.421250,-0.443614,-0.441399,-0.083431,-0.087286,0.036600,...,-0.607536,2.095085,-0.297182,0.757568,1.235382,0.536618,-0.880668,-0.245788,-0.225857,0
4,1.093672,2.372354,1.544901,-0.407317,0.749587,1.050296,1.074624,-0.083431,-0.087286,0.357676,...,-0.607536,-0.477308,-0.297182,-0.885894,-1.172618,-1.370928,-0.880668,-0.245788,-0.658635,1


In [342]:
X = df.drop(columns=['y'])
y = df[['y']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# Classification Neural Network

In [343]:
X_train_tensor = torch.tensor(X_train.values).to(torch.float32)
X_test_tensor = torch.tensor(X_test.values).to(torch.float32)
y_train_tensor = torch.tensor(y_train.values).to(torch.float32)
y_test_tensor = torch.tensor(y_test.values).to(torch.float32)

In [344]:
class MyNeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer1 = nn.Linear(77, 500)
    self.relu1 = nn.ReLU()

    self.layer2 = nn.Linear(500, 200)
    self.relu2 = nn.ReLU()

    self.layer3 = nn.Linear(200, 500)
    self.relu3 = nn.ReLU()

    self.layer4 = nn.Linear(500, 50)
    self.relu4 = nn.ReLU()

    self.layer5 = nn.Linear(50, 1)


    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.layer1(x)
    x = self.relu1(x)

    x = self.layer2(x)
    x = self.relu2(x)

    x = self.layer3(x)
    x = self.relu3(x)

    x = self.layer4(x)
    x = self.relu4(x)

    x = self.layer5(x)

    pred = self.sigmoid(x)
    return pred

In [345]:
model = MyNeuralNetwork()

loss_fn = nn.BCELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [346]:
model.train()

for epoch in range(10):
  for i in range(0, len(X_test), 1):
    pred = model(X_train_tensor[i:i+1])
    loss = loss_fn(pred, y_train_tensor[i:i+1])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

In [347]:
model.eval()

# [tp, tn, fp, fn]
errors = [0, 0, 0, 0]

for i in range(len(X_test)):
  with torch.no_grad():
    y_pred = model(X_test_tensor[i]).round()

    if y_pred == y_test_tensor[i] and y_pred == 1:
      # TP
      errors[0] += 1
    elif y_pred == y_test_tensor[i] and y_pred == 0:
      # TN
      errors[1] += 1
    elif y_pred != y_test_tensor[i] and y_pred == 1:
      # FP
      errors[2] += 1
    elif y_pred != y_test_tensor[i] and y_pred == 0:
      # FN
      errors[3] += 1

In [348]:
from sklearn.metrics import f1_score

accuracy = (errors[0] + errors[1]) / sum(errors)
precision = (errors[0]) / (errors[0] + errors[2])
recall = (errors[0]) / (errors[0] + errors[3])

# F1 Score = 2 x (Precision x Recall)/(Precision + Recall)
F1 = (2 * precision * recall) / (precision + recall)


print('Accuracy: ', accuracy)
print('Precision: ', precision)
print('Recall: ', recall)
print('F1: ', F1)

Accuracy:  0.58375
Precision:  0.5604133545310016
Recall:  0.7160995429151854
F1:  0.6287625418060201


In [ ]:
# Download the pth file
from google.colab import files

torch.save(model.state_dict(), 'Classification-model.pth')
files.download('Classification-model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>